# Filter inconcistent writing

In [ ]:
import pandas as pd

df = pd.read_csv('./datasets/raw/tdec_raw.csv')

In [ ]:
cek_t = df['body'].str.contains('^TEMPO')
cek_d = df['body'].str.contains(' - ')
df = df[cek_t & cek_d]
df['body'] = df['body'].apply(lambda s: s.split(' - ')[1])

In [ ]:
df.to_csv('./datasets/raw/tdec.csv', index=False)

# Split dataset for LDA and BERTopic

In [ ]:
import pandas as pd

df = pd.read_csv('./datasets/raw/tdec.csv')

In [ ]:
import random

def generate_types(df):
    l = len(df)
    ltv = int(l * 0.15)
    lt = l - 2 * ltv
    types = ['train'] * lt + ['test'] * ltv + ['val'] * ltv
    random.shuffle(types)
    return types

df['type'] = generate_types(df)

In [ ]:
df_short_lda = df.drop('body', axis=1)
df_short_bert = df.drop('body', axis=1)

df_long_lda = df.drop('headline', axis=1)
df_long_bert = df.drop('headline', axis=1)
df_long_lda.columns = ['article', 'type']
df_long_bert.columns = ['article', 'type']
df_long_lda['article'] = df['headline'] + '. ' + df['body']
df_long_bert['article'] = df['headline'] + '. ' + df['body']

In [ ]:
# df_short_bert.to_csv('./datasets/bert_short/corpus.tsv', sep='\t', index=False, header=None)
# df_long_bert.to_csv('./datasets/bert_long/corpus.tsv', sep='\t', index=False, header=None)

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import string

df_sw = pd.read_csv('./datasets/stopwords-id.csv')
stem = StemmerFactory().create_stemmer().stem

translation = str.maketrans('', '', string.punctuation)
sw = list(df_sw['stopword'])
def preprocess(s):
    clean = s.translate(translation)
    stemmed = stem(clean)
    swed = ' '.join([w for w in stemmed.split(' ') if not w in sw])
    return swed

In [ ]:
df_short_lda['headline'] = df_short_lda['headline'].apply(preprocess)
df_long_lda['article'] = df_long_lda['article'].apply(preprocess)

In [ ]:
# df_short_lda.to_csv('./datasets/lda_short/corpus.tsv', sep='\t', index=False, header=None)
# df_long_lda.to_csv('./datasets/lda_long/corpus.tsv', sep='\t', index=False, header=None)

In [2]:
import pandas as pd

df_short_lda = pd.read_csv('./datasets/lda_short/corpus.tsv', sep='\t', names=['headline', 'type'])
df_long_lda = pd.read_csv('./datasets/lda_long/corpus.tsv', sep='\t', names=['article', 'type'])

In [3]:
from gensim.corpora.dictionary import Dictionary

df_short_lda_vocab = Dictionary(df_short_lda['headline'].apply(lambda s: s.split(' ')))
df_long_lda_vocab = Dictionary(df_long_lda['article'].apply(lambda s: s.split(' ')))

In [4]:
df_short_lda_vocab = pd.DataFrame(df_short_lda_vocab.token2id.keys())
df_long_lda_vocab = pd.DataFrame(df_long_lda_vocab.token2id.keys())

In [7]:
df_short_lda_vocab.to_csv('./datasets/lda_short/vocabulary.txt', index=False, header=None)
df_long_lda_vocab.to_csv('./datasets/lda_long/vocabulary.txt', index=False, header=None)